In [1]:
from api_token import token
import todoist as td
import pandas as pd
import datetime as dt

In [2]:
# Get yesterday's date to feed into call
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
yday_start = str(yesterday)+"T00:00:00Z"
yday_end = str(yesterday)+"T23:59:59Z"
print(yday_start)
print(yday_end)

2021-01-08T00:00:00Z
2021-01-08T23:59:59Z


In [3]:
api = td.TodoistAPI(token)
# Sync to API to get latest data
api.sync();

In [4]:
yday_completed_tasks_dict = api.completed.get_all(since = yday_start, until = yday_end)

In [85]:
# get list of keys
[key for key in yday_completed_tasks_dict]

['items', 'projects']

In [86]:
yday_completed_tasks_df = pd.DataFrame(yday_completed_tasks_dict['items'])

In [102]:
# our df doesn't yet have project name
# create new dict of id and project name
td_projects_dict = yday_completed_tasks_dict['projects']
projects = {int(key):td_projects_dict[key]['name'] for key in td_projects_dict}

In [89]:
# map project names to df
yday_completed_tasks_df['project_name'] = yday_completed_tasks_df['project_id'].map(projects)

In [90]:
# TODO to get task url you have to create it as follows https://todoist.com/showTask?id={task_id}
yday_completed_tasks_df['task_url'] = yday_completed_tasks_df['task_id'].map('https://todoist.com/showTask?id={}'.format)

In [91]:
# Drop unwanted columns
yday_completed_tasks_df = yday_completed_tasks_df.drop(columns = ['id', 'meta_data', 'project_id', 'user_id', 'task_id'])

In [ ]:
# send email

# import yagmail

# receiver = "your@gmail.com"
# body = "Hello there from Yagmail"
# filename = "document.pdf"

# yag = yagmail.SMTP("my@gmail.com")
# yag.send(
#     to=receiver,
#     subject="Yagmail test with attachment",
#     contents=body, 
#     attachments=filename,
# )

In [97]:
# from send_email import send_mail
# from get_data import get_gdp_data
from pretty_html_table import build_table


def prettify_data(data):
#     gdp_data = get_gdp_data()
    output = build_table(data, 'blue_light')
#     send_mail(output)
    return output


# prettify_data(yday_completed_tasks_df)

In [103]:
# import yagmail
# yag = yagmail.SMTP("nibe0993")
# contents = [
#     "Testing \n", prettify_data(yday_completed_tasks_df) ]
# yag.send('nibe12@live.com', 'subject', contents)

In [29]:
# Order of columns ["task_content_description", "project_name", "completed_at"
# , "labels", "priority", "task_link_url", "due_date"]

# TODOs
# add column with label names
# add column with project name
# add column with url link